In [4]:
# Import necessary libraries
import os
import warnings
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
import itertools
from sklearn import datasets
from scipy.optimize import fsolve
import matplotlib.gridspec as gridspec
from matplotlib.colors import ListedColormap
from sklearn.metrics import pairwise_distances
from sklearn.isotonic import IsotonicRegression
from metrics import *
import plotly.graph_objects as go


from pypalettes import load_cmap
cmap = load_cmap("Austria")
cmap = cmap.colors

rrange = np.linspace(0, 12, 1200)

norm_stress_scores = list()

# Create figure
fig = go.Figure()

X = np.load("datasets/iris.npy")
for (technique, llabel, color) in zip(["MDS", "TSNE", "RANDOM"], ["MDS", "t-SNE", "Random"], [cmap[0], cmap[1], cmap[2]]):
    Y = np.load(f"embeddings/iris_{technique}_1.npy")

    M = Metrics(X, Y)

    scale_opt, alpha_opt = M.compute_scale_normalized_stress(return_alpha=True)

    for alpha in rrange:
        M.setY(alpha * Y)
        norm_stress_scores.append(M.compute_normalized_stress())

    # Add a trace for each technique
    fig.add_trace(go.Scatter(x=rrange, y=norm_stress_scores,
                  mode='lines', name=llabel, line=dict(color=color[:-2])))

    # Add a marker for the minimum point on each curve
    fig.add_trace(go.Scatter(x=[alpha_opt], y=[
                  scale_opt], mode='markers', name=f"{llabel} Minimum", marker=dict(color=color[:-2])))

    norm_stress_scores.clear()

fig.update_layout(
    title="Iris",
    xaxis_title="Scale value",
    yaxis_title="log(Normalized stress)",
    yaxis_type="log",
    autosize=False,
    width=800,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=4)
)

fig.update_layout(template='simple_white')
fig.update_yaxes(tick0=0, dtick=1)

# Save the figure as HTML
fig.write_html("./webpage/index.html", config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['zoom2d', 'pan2d', 'select2d', 'lasso2d', 'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian', 'toggleSpikelines', 'toImage']
})